In [13]:
%pip install timm torch torchvision opencv-python matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 24.8 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


### 환경변수 설정 및 라이브러리 임포트

In [12]:
# 1. 자동 리로드 확장 도구 로드
%load_ext autoreload
# 2. 모든 모듈을 실행할 때마다 다시 불러오도록 설정
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import os
import sys
import json
import time
from dotenv import load_dotenv, find_dotenv

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from prepare_l3 import prepare_l3_samplings, l2_bridge_binary
from Generate import generate_l3

home = os.path.expanduser("~")
load_dotenv(find_dotenv())

API_KEY = os.getenv("Gemini_API_KEY")

SRC_DATA_ROOT = os.path.join(home, "Desktop", "PlantVillage")
WORK_DIR = os.path.join(home, "Desktop", "GLaMM", "datasets", "PlantVillage", "L3_Work_Dir")
OUTPUT_DIR = os.path.join(home, "Desktop", "GLaMM", "datasets", "PlantVillage", "L3_Final_Output")
JSONL_OUTPUT_PATH = os.path.join(OUTPUT_DIR, "l3_plant_village_1200.jsonl")

os.makedirs(os.path.join(OUTPUT_DIR, "masks/l3"), exist_ok=True)

print("환경변수 설정 및 라이브러리 임포트 성공")

환경변수 설정 및 라이브러리 임포트 성공


### Gemini API 및 MobileSAM 설정

In [14]:
model, predictor = generate_l3.init_models(API_KEY)
print("Gemini API 및 MobileSAM 설정 완료")

Gemini API 및 MobileSAM 설정 완료


### PlantVillage 데이터셋 샘플링
`Healthy`류(3개 클래스): 200장,
`unhealthy`류(12개 클래스): 50장

In [7]:
prepare_l3_samplings(SRC_DATA_ROOT, WORK_DIR)
print(f"{WORK_DIR}에 샘플링 데이터 준비 완료.")

 Potato___Early_blight: 50장 복사완료 [Unhealthy (50)]
 Tomato___Spider_mites_Two_spotted_spider_mite: 50장 복사완료 [Unhealthy (50)]
 Tomato___Target_Spot: 50장 복사완료 [Unhealthy (50)]
 Potato___Late_blight: 50장 복사완료 [Unhealthy (50)]
 Tomato___Late_blight: 50장 복사완료 [Unhealthy (50)]
 Tomato___Tomato_mosaic_virus: 50장 복사완료 [Unhealthy (50)]
 Tomato___Leaf_Mold: 50장 복사완료 [Unhealthy (50)]
 Tomato___Bacterial_spot: 50장 복사완료 [Unhealthy (50)]
 Tomato___Tomato_YellowLeaf_Curl_Virus: 50장 복사완료 [Unhealthy (50)]
 Tomato___Early_blight: 50장 복사완료 [Unhealthy (50)]
 Tomato___healthy: 200장 복사완료 [Healthy (200)]
 Pepper_bell___healthy: 200장 복사완료 [Healthy (200)]
 Tomato___Septoria_leaf_spot: 50장 복사완료 [Unhealthy (50)]
 Pepper_bell___Bacterial_spot: 50장 복사완료 [Unhealthy (50)]
 Potato___healthy: 152장 복사완료 [Healthy (200)]

총 1152장의 이미지가 샘플링됨
/Users/bosung/Desktop/GLaMM/datasets/PlantVillage/L3_Work_Dir에 샘플링 데이터 준비 완료.


### 데이터 생성 준비 및 클래스 스캔

In [5]:
all_classes = [d for d in os.listdir(WORK_DIR) if os.path.isdir(os.path.join(WORK_DIR, d))]
l3_final_dataset = []

print(f"처리할 클래스 목록: {all_classes}")
print(f"총 {len(all_classes)}개 클래스 확인")

처리할 클래스 목록: ['Potato___Early_blight', 'Tomato___Spider_mites_Two_spotted_spider_mite', 'Tomato___Target_Spot', 'Potato___Late_blight', 'Tomato___Late_blight', 'Tomato___Tomato_mosaic_virus', 'Tomato___Leaf_Mold', 'Tomato___Bacterial_spot', 'Tomato___Tomato_YellowLeaf_Curl_Virus', 'Tomato___Early_blight', 'Tomato___healthy', 'Pepper_bell___healthy', 'Tomato___Septoria_leaf_spot', 'Pepper_bell___Bacterial_spot', 'Potato___healthy']
총 15개 클래스 확인


### Level-3 데이터 생성

In [17]:
start_time = time.time()

for idx, folder_name in enumerate(all_classes):
    folder_path = os.path.join(WORK_DIR, folder_name)
    images = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    print(f"[{idx+1}/{len(all_classes)}] {folder_name} 처리 중 (이미지 {len(images)}장)")

    for img_name in images:
        img_rel_path = os.path.join(folder_name, img_name)
        
        # 2~3단계: 데이터별 Bridge 생성 (질문 및 프롬프트 동적 주입)
        data_sample = l2_bridge_binary(img_rel_path, folder_name)
        
        # 4단계: generate_l3 호출 (Gemini 추론 + MobileSAM 마스킹)
        l3_result = generate_l3.generate_l3(
            image_dir=WORK_DIR, 
            output_dir=OUTPUT_DIR, 
            l2_data_sample=data_sample, 
            prompt=data_sample['prompt'], 
            model=model, 
            predictor=predictor
        )
        
        if l3_result:
            l3_final_dataset.append(l3_result)
            
        # API 할당량 및 과열 방지를 위한 미세한 지연
        time.sleep(0.5)

print(f"생성 완료! 현재 생성된 데이터 개수: {len(l3_final_dataset)}")

[1/15] Potato___Early_blight 처리 중 (이미지 50장)
[2/15] Tomato___Spider_mites_Two_spotted_spider_mite 처리 중 (이미지 50장)
[3/15] Tomato___Target_Spot 처리 중 (이미지 50장)
L3 생성 실패: Expecting value: line 1 column 1 (char 0)
[4/15] Potato___Late_blight 처리 중 (이미지 50장)
[5/15] Tomato___Late_blight 처리 중 (이미지 50장)
[6/15] Tomato___Tomato_mosaic_virus 처리 중 (이미지 50장)
L3 생성 실패: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting
L3 생성 실패: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting
[7/15] Tomato___Leaf_Mold 처리 중 (이미지 50장)
L3 생성 실패: unsupported operand type(s) for /: 'list' and 'int'
[8/15] Tomato___Bacterial_spot 처리 중 (이미지 50장)
L3 생성 실패: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting
L3 생성 실패: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.goog

In [8]:
import google.generativeai as genai
import os

# API 설정 확인
genai.configure(api_key=os.getenv("Gemini_API_KEY"))

# 사용 가능한 모델 출력
print("📋 사용 가능한 모델 목록:")
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(f"- {m.name}")

📋 사용 가능한 모델 목록:
- models/gemini-2.5-flash
- models/gemini-2.5-pro
- models/gemini-2.0-flash-exp
- models/gemini-2.0-flash
- models/gemini-2.0-flash-001
- models/gemini-2.0-flash-exp-image-generation
- models/gemini-2.0-flash-lite-001
- models/gemini-2.0-flash-lite
- models/gemini-2.0-flash-lite-preview-02-05
- models/gemini-2.0-flash-lite-preview
- models/gemini-exp-1206
- models/gemini-2.5-flash-preview-tts
- models/gemini-2.5-pro-preview-tts
- models/gemma-3-1b-it
- models/gemma-3-4b-it
- models/gemma-3-12b-it
- models/gemma-3-27b-it
- models/gemma-3n-e4b-it
- models/gemma-3n-e2b-it
- models/gemini-flash-latest
- models/gemini-flash-lite-latest
- models/gemini-pro-latest
- models/gemini-2.5-flash-lite
- models/gemini-2.5-flash-image
- models/gemini-2.5-flash-preview-09-2025
- models/gemini-2.5-flash-lite-preview-09-2025
- models/gemini-3-pro-preview
- models/gemini-3-flash-preview
- models/gemini-3-pro-image-preview
- models/nano-banana-pro-preview
- models/gemini-robotics-er-1.5-pre

In [18]:
import json
import os

# 저장 경로 및 파일 이름 설정
save_dir = "/Users/bosung/Desktop/GLaMM/datasets/PlantVillage"
save_filename = "l3_plant_village_metadata.json"
save_path = os.path.join(save_dir, save_filename)

# 폴더가 없을 경우 생성
os.makedirs(save_dir, exist_ok=True)

# JSON 파일로 저장
try:
    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(l3_final_dataset, f, ensure_ascii=False, indent=4)
    
    print(f"저장된 데이터 개수: {len(l3_final_dataset)}개")

except Exception as e:
    print(f" 저장 실패: {e}")

저장된 데이터 개수: 1144개
